In [ ]:
# script to replace the base mechanism with new reactions
# Includes every reaction and species calculated above the uncertainty threshold

In [ ]:
import os
import subprocess
import numpy as np

import rmgpy.chemkin
import rmgpy.data.kinetics


import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# Load the base model
basedir = '../models/base_rmg'
base_chemkin = os.path.join(basedir, 'chem_annotated.inp')
dictionary = os.path.join(basedir, 'species_dictionary.txt')
transport = os.path.join(basedir, 'tran.dat')
species_list, reaction_list = rmgpy.chemkin.load_chemkin_file(base_chemkin, dictionary_path=dictionary, transport_path=transport)
print(f'{len(species_list)} species, {len(reaction_list)} reactions')

# load the uncorrelated uncertainty
kinetic_uncertainty = np.load(os.path.join(basedir, 'kinetic_uncertainty.npy'))
thermo_uncertainty = np.load(os.path.join(basedir, 'thermo_uncertainty.npy'))

In [ ]:
# Load the new kinetics library
kinetics_lib = '../kinetics'
ark_kinetics_database = rmgpy.data.kinetics.KineticsDatabase()
ark_kinetics_database.load_libraries(kinetics_lib)
print(f'{len(ark_kinetics_database.libraries[""].entries)} kinetic entries loaded')


# Load the new thermo library
thermo_lib = '../thermo'
ark_thermo_database = rmgpy.data.thermo.ThermoDatabase()
ark_thermo_database.load_libraries(thermo_lib)
print(f'{len(ark_thermo_database.libraries["harris_butane"].entries)} thermo entries loaded')

In [ ]:
# Figure out which reactions to change
uncertainty_cutoff = 3.0
change_list = []
assert len(kinetic_uncertainty) == len(reaction_list)
for i in range(0, len(reaction_list)):
    if kinetic_uncertainty[i] > uncertainty_cutoff:
        change_list.append(i)

In [ ]:
# Change the kinetics for the specified reactions
total_changed = 0
for key in ark_kinetics_database.libraries[''].entries.keys():
    entry = ark_kinetics_database.libraries[''].entries[key]
    index = entry.index
    for i in range(0, len(reaction_list)):
        if entry.item.is_isomorphic(reaction_list[i]):
            rmg_rxn_index = i
            break
    else:  
        print(f'could not match reaction index {index}')
        continue
    
    if rmg_rxn_index in change_list:
        print(f'changing reaction {rmg_rxn_index} {reaction_list[rmg_rxn_index]}')
        reaction_list[rmg_rxn_index].kinetics = entry.data
        total_changed += 1

print(total_changed, 'changed')

In [ ]:
# Figure out which species to change
uncertainty_cutoff = 3.0
thermo_change_list = []
assert len(thermo_uncertainty) == len(species_list)
for i in range(0, len(species_list)):
    if thermo_uncertainty[i] > uncertainty_cutoff:
        thermo_change_list.append(i)

In [ ]:
# stitch each of the library thermo params into the mechanism

sp_changed = 0
for key in ark_thermo_database.libraries['harris_butane'].entries.keys():
    entry = ark_thermo_database.libraries['harris_butane'].entries[key]
    for i, species in enumerate(species_list):
        if entry.item.smiles == species.smiles:
            rmg_species_index = i
            break
    else:
        entry_sp = rmgpy.species.Species(smiles=entry.item.smiles)
        for j, species in enumerate(species_list):
            if entry_sp.is_isomorphic(species.molecule[0]):
                rmg_species_index = i
                'matched'
                break
        else:
            continue
            raise ValueError (f'could not match species index {entry}')
        
        
    if rmg_species_index in thermo_change_list:
        print(f'Changing estimated thermo {rmg_species_index}: {species_list[rmg_species_index]}')
        species_list[rmg_species_index].thermo = entry.data
        
        sp_changed += 1
    
print(sp_changed, 'changed')   
    

In [ ]:
# Save the new mechanism
chemkin_file = os.path.join(f'cutoff3_20221205.inp')
rmgpy.chemkin.save_chemkin_file(chemkin_file, species_list, reaction_list, verbose=True, check_for_duplicates=True)
subprocess.run(['ck2cti', f'--input={chemkin_file}', f'--transport={transport}', f'--output={chemkin_file[:-4]}.cti'])
